In [13]:
# Import relevant libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from zipfile import ZipFile

import mne
import mne_nirs

from mne_nirs.experimental_design import make_first_level_design_matrix
from mne_nirs.statistics import run_glm

# %%
# Import relevant libraries
import os #computer stuff
import numpy as np
import matplotlib.pyplot as plt #plots
import pandas as pd #to work with datasets
from zipfile import ZipFile
import os

import mne
import mne_nirs

# Import StatsModels
import statsmodels.formula.api as smf
import tqdm


from mne_nirs.experimental_design import make_first_level_design_matrix #making design matrices
from mne_nirs.statistics import run_glm #making ocntrasts
from mne_nirs.statistics import statsmodels_to_results
from mne_nirs.visualisation import plot_glm_group_topo
from mne_nirs.visualisation import plot_glm_surface_projection
from mne_nirs.channels import (get_long_channels, 
                               get_short_channels,
                               picks_pair_to_idx) 
from mne.preprocessing.nirs import optical_density, beer_lambert_law

from nilearn.plotting import plot_design_matrix #another project before mne nirs, they are using it for plotting design matrices

from itertools import compress #for loops, iterations in general
from icecream import ic #like "print"

# Import MNE-BIDS processing
from mne_bids import BIDSPath, read_raw_bids, get_entity_vals

import matplotlib as mpl
from lets_plot import *
LetsPlot.setup_html()

from plotnine import ggplot, aes, geom_line, geom_point, facet_grid, geom_hline

print('packages have been imported')

packages have been imported


That was painful. Remember to check that you are installing the packages into the specific python environment (might have to use the terminal outside of VSC). Here, I use Python 3.9.7. 
Continuing to the preprocessing.

First, we have some clever functions. 

In [14]:

def normalize(data, train_split):
    """
    Normalize the data
    """
    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)
    return (data - data_mean) / data_std


def preprocess(raw_intensity, verbose=True):
    """
    Loads the raw data from the path and procesess it
    """
    # if verbose:
    #     ic("Loading ", path)
    # raw_intensity = mne.io.read_raw_snirf(raw_path, preload=True)
    raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)

    # sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od, l_freq=0.7, h_freq=1.5)
    # raw_od.info['bads'] = list(compress(raw_od.ch_names, sci < 0.5))

    # if verbose:
    ic("Apply short channel regression.")
    od_corrected = mne_nirs.signal_enhancement.short_channel_regression(raw_od)

    if verbose:
        ic("Do temporal derivative distribution repair on:", od_corrected)
    tddr_od = mne.preprocessing.nirs.tddr(od_corrected)

    if verbose:
        ic("Convert to haemoglobin with the modified beer-lambert law.")
    raw_haemo = beer_lambert_law(tddr_od, ppf=0.1)

    if verbose:
        ic("Apply further data cleaning techniques and extract epochs.")
    raw_haemo = mne_nirs.signal_enhancement.enhance_negative_correlation(
        raw_haemo)

    # if verbose:
    #     ic("Separate the long channels and short channels.")
    # sht_chans = get_short_channels(raw_haemo)
    # raw_haemo = get_long_channels(raw_haemo)

    # if verbose:
    #     ic("Bandpass filter on:", raw_haemo)
    # filter_haemo = raw_haemo.filter(
    #     0.01, 0.7, h_trans_bandwidth=0.3, l_trans_bandwidth=0.005)

    # Create a design matrix
    # design_matrix = make_first_level_design_matrix(raw_haemo)

    # Append short channels mean to design matrix
    # design_matrix["ShortHbO"] = np.mean(
    #     sht_chans.copy().pick(picks="hbo").get_data(), axis=0)
    # design_matrix["ShortHbR"] = np.mean(
    #     sht_chans.copy().pick(picks="hbr").get_data(), axis=0)

    return raw_haemo

def individual_analysis(bids_path, ID, roi_1=[[4, 3], [1, 3], [3, 3], [1, 2], [2, 3], [1, 1]], roi_2=[[8, 7], [5, 7], [7, 7], [5, 6], [6, 7], [5, 5]], contrasts_1=['76','77'],contrasts_2=['78','79']):

    # raw_intensity = read_raw_bids(bids_path=bids_path, verbose=False)
    raw_intensity = mne.io.read_raw_snirf(bids_path)

    raw_haemo = preprocess(raw_intensity)

    # Cut out just the short channels for creating a GLM repressor
    sht_chans = get_short_channels(raw_haemo)
    raw_haemo = get_long_channels(raw_haemo)

    # Create a design matrix
    design_matrix = make_first_level_design_matrix(raw_haemo)

    # Append short channels mean to design matrix
    design_matrix["ShortHbO"] = np.mean(sht_chans.copy().pick(picks="hbo").get_data(), axis=0)
    design_matrix["ShortHbR"] = np.mean(sht_chans.copy().pick(picks="hbr").get_data(), axis=0)

    # Run GLM
    glm_est = run_glm(raw_haemo, design_matrix)

    # Define channels in each region of interest
    # List the channel pairs manually
    # Then generate the correct indices for each pair
    groups = dict(
        ROI_1=picks_pair_to_idx(raw_haemo, roi_1, on_missing='ignore'),
        ROI_2=picks_pair_to_idx(raw_haemo, roi_2, on_missing='ignore'))

    # Extract channel metrics
    cha = glm_est.to_dataframe()

    # Compute region of interest results from channel data
    roi = glm_est.to_dataframe_region_of_interest(groups,
                                                  design_matrix.columns,
                                                  demographic_info=True)

    # Define left vs right tapping contrast
    contrast_matrix = np.eye(design_matrix.shape[1])
    basic_conts = dict([(column, contrast_matrix[i])
                        for i, column in enumerate(design_matrix.columns)])

    contrast_LvR = basic_conts[contrasts_1[0]]
    for idx, con in enumerate(contrasts_1):
        if idx != 0: contrast_LvR + basic_conts[con]
    for con in contrasts_2:
        contrast_LvR - basic_conts[con]
    
    # Compute defined contrast
    contrast = glm_est.compute_contrast(contrast_LvR)
    con = contrast.to_dataframe()

    # Add the participant ID to the dataframes
    roi["ID"] = cha["ID"] = con["ID"] = ID

    # Convert to uM for nicer plotting below.
    cha["theta"] = [t * 1.e6 for t in cha["theta"]]
    roi["theta"] = [t * 1.e6 for t in roi["theta"]]
    con["effect"] = [t * 1.e6 for t in con["effect"]]

    return raw_haemo, roi, cha, con


Next, we put in our information. That is, which files to preprocess, the regions of interest and contrasts. 

In [1]:
df_roi = pd.DataFrame()  # To store region of interest results
df_cha = pd.DataFrame()  # To store channel level results
df_con = pd.DataFrame()  # To store channel level contrast results

subjects = [
    "/Users/al/Library/Mobile Documents/com~apple~CloudDocs/Cognitive Science 🧠/Bachelor's Thesis/Data/NCPE-2021-11-01/NP-Ph2-305-2021-11-01/2021-11-01_003.snirf",
    "/Users/al/Library/Mobile Documents/com~apple~CloudDocs/Cognitive Science 🧠/Bachelor's Thesis/Data/NCPE-2021-11-04/NP-Ph2-306-2021-11-04/2021-11-04_001.snirf"
]

for idx, sub in enumerate(subjects):
    # Analyse data and return both ROI and channel results
    raw_haemo, roi, channel, con = individual_analysis(sub, idx, roi_1=[[1,1], [1,2]], roi_2=[[4,4], [4,5]], contrasts_1=['76', '77'], contrasts_2=['78', '72'])

    # Append individual results to all participants
    df_roi = df_roi.append(roi)
    df_cha = df_cha.append(channel)
    df_con = df_con.append(con)

i'm on a roll


To the good part

In [ ]:
grp_results = df_roi.query("Condition in ['76', '77', '78']")
grp_results = grp_results.query("Chroma in ['hbo']")

grp_results.plot(x = "Condition", y = "theta")

grp_results

Modelling time


In [ ]:
grp_results = df_roi.query("Condition in ['76', '77', '78']")

roi_model = smf.mixedlm("theta ~ -1 + ROI:Condition:Chroma",
                        grp_results, groups=grp_results["ID"]).fit(method='nm')
roi_model.summary()

Plotting time

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
con_summary = df_con.query("Chroma in ['hbo']")

# Run group level model and convert to dataframe
con_model = smf.mixedlm("effect ~ -1 + ch_name:Chroma",
                        con_summary, groups=con_summary["ID"]).fit(method='nm')
con_model_df = statsmodels_to_results(con_model,
                                      order=raw_haemo.copy().pick(
                                          picks="hbo").ch_names)

con_model_df['Coef.'] = -con_model_df['Coef.']

plot_glm_group_topo(raw_haemo.copy().pick(picks="hbo"),
                    con_model_df, colorbar=True, axes=axes)

2D plotting

In [ ]:
# Generate brain figure from data
clim = dict(kind='value', pos_lims=(0, 8, 11))
brain = plot_glm_surface_projection(raw_haemo.copy().pick("hbo"),
                                    con_model_df, clim=clim, view='dorsal',
                                    colorbar=True, size=(800, 700))
brain.add_text(0.05, 0.95, "Left-Right", 'title', font_size=16, color='k')

# Run model code as above
clim = dict(kind='value', pos_lims=(0, 11.5, 17))
for idx, cond in enumerate(['76', '78']):

    # Run same model as explained in the sections above
    ch_summary = df_cha.query("Condition in [@cond]")
    ch_summary = ch_summary.query("Chroma in ['hbo']")
    ch_model = smf.mixedlm("theta ~ -1 + ch_name", ch_summary,
                           groups=ch_summary["ID"]).fit(method='nm')
    model_df = statsmodels_to_results(ch_model, order=raw_haemo.copy().pick("hbo").ch_names)

    # Generate brain figure from data
    brain = plot_glm_surface_projection(raw_haemo.copy().pick("hbo"),
                                        model_df, clim=clim, view='dorsal',
                                        colorbar=True, size=(800, 700))
    brain.add_text(0.05, 0.95, cond, 'title', font_size=16, color='k')

something else

In [ ]:
raw_haemo = preprocess(mne.io.read_raw_snirf(raw_path))
events, event_dict = mne.events_from_annotations(raw_haemo, verbose=False)
mne.viz.plot_events(events, event_id=event_dict, sfreq=raw_haemo.info['sfreq'])

s = mne_nirs.experimental_design.create_boxcar(raw_haemo)
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(15, 6))
plt.plot(raw_haemo.times, s, axes=axes)
plt.xlabel("Time (s)")